<img src="https://upload.wikimedia.org/wikipedia/en/thumb/4/48/Hoare_Lea_logo.svg/1280px-Hoare_Lea_logo.svg.png" width=300 />

**Pablo Arango | Senior Computational Developer**

[pabloarango@hoarelea.com](mailto:pabloarango@hoarelea.com)

---

# Wind Data Post-processing from Hourly Data to MET office Frequency format
### Goal
The main goal of this workflows is to translate hourly 20 year data purchased in the MET office and translated into an fre aggregate the individual results from ANSYS CFX into the following databases:

### Inputs
The inputs for this workflow are:
- `WEATHER_DATA` - Path to the Excel weather data received from the MET office.
- `TEMPLATE_FILE` - Path to the template Excel file

### Outputs
- `export-weather-data.xlsx` - Excel file following MET office format.

## Inputs

In [30]:
WEATHER_DATA = '20-Year-Data-Northolt.xlsx'
TEMPLATE_FILE = 'Template_WFA.xlsx'

## Import main libraries and install libraries
Currently this scripts relies in `xlwings` that directly uses Excel, hence there is no compatibility with Linux systems (eg. Google Colab)

In [31]:
!pip install pandas numpy xlwings 

In [32]:
import numpy as np
import xlwings as xw
import pandas as pd
import shutil

In [48]:
def read_data(WEATHER_DATA):
    # Read data from the excel file
    df = pd.read_excel(WEATHER_DATA, skiprows=10, header=0)
    # rename columns
    df.columns = ['date','air_temp','wet_temp','rel_hum','wind_speed', 'wind_dir']
    # Convert date to datetime
    df['date'] = pd.to_datetime(df['date'])
    # drop rows with NaN values on wind_dir
    df = df.dropna(subset=['wind_dir'])
    return df

def calc_wind_dir_bin(df):
    # Create wind direction bins that match the MET office format
    labels = [0, 1, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330] # note that 0 and 1 are the same and would be corrected in the next step
    bins = pd.IntervalIndex.from_tuples([(345, 360), (-0.1, 15), (15, 45), (45,75), (75,105), (105,135), (135,165), (165,195), (195,225), (225,255), (255,285), (285,315), (315,345)] )
    
    # calculate frequency of each wind speed from 0 to 50 m/s every 1 m/s. after 50 m/s, goes from 50 to 999 m/s.
    wind_bin = pd.cut(df.wind_dir, bins=bins)
    wind_bin = wind_bin.cat.rename_categories(labels)
    wind_bin = wind_bin.astype('int').replace(1, 0)
    print(wind_bin.unique())
    return wind_bin
    

def add_margins(df):
    df.loc['ALL OBS']= df.sum(numeric_only=True, axis=0)
    df.loc[:,'ALL OBS'] = df.sum(numeric_only=True, axis=1)
    return df


def frequency_tables(df):
    # CREATE FREQUENCY TABLES
    df_freq = pd.crosstab(df.wind_speed_round, df.wind_dir_bin).reindex(range(0, 53)).fillna(0)
    df_freq = add_margins(df_freq)
    df_freq = df_freq.astype('int')
    df_freq.loc[52] = np.nan # create empty row at index 52
    
    
    # CREATE FREQUENCY TABLES NORMALISED
    df_freq_norm = pd.crosstab(df.wind_speed_round, df.wind_dir_bin, normalize='all').reindex(range(0, 53)).fillna(0)
    df_freq_norm = add_margins(df_freq_norm)
    df_freq_norm = (df_freq_norm * 100).round(1)
    df_freq_norm.loc[52] = np.nan # create empty row at index 52
    
    return df_freq, df_freq_norm


def read_header(WEATHER_DATA):
    # read header from data file 
    header = pd.read_excel(WEATHER_DATA, nrows=8, header=None).iloc[:,:2] # read only the first two columns
    header = header.set_index(0).T
    
    # search text in Start and End columns after 4 digits 
    start = header['Start:'].str.extract(r'\d{4}(.*)').values[0][0]
    end = header['End:'].str.extract(r'\d{4}(.*)').values[0][0]
    header['Period'] = f"{start} to {end}"
    
    header_str = f"""{header.Station.str.upper().values[0]}
    LAT. {header['Latitude'].values[0]} 
    LONG. {header['Longtude'].values[0]}
    ALT. {header['Altitude (m)'].values[0]}
    Period: {header['Period'].values[0]}"""
    
    header_lst = header_str.splitlines()
    header_lst = [x.strip() for x in header_lst]
    
    return header_lst

In [46]:
df = read_data(WEATHER_DATA)
df.head()

,date,air_temp,wet_temp,rel_hum,wind_speed,wind_dir
0,2002-08-01 00:00:00,14.4,13.6,91.0,1.5,200.0
1,2002-08-01 01:00:00,13.3,12.8,94.2,0.5,190.0
2,2002-08-01 02:00:00,13.0,12.7,96.5,0.5,200.0
3,2002-08-01 03:00:00,12.8,12.5,96.4,0.0,0.0
4,2002-08-01 04:00:00,12.4,12.2,97.6,0.0,0.0


In [35]:
df.wind_dir.isna().sum() == 0 # check if there are any NaN values

True

In [49]:
wind_bin = calc_wind_dir_bin(df)

[210 180   0 240  60  30  90 120 150 300 270 330]


In [50]:
df['wind_dir_bin'] = wind_bin
df['month'] = df.date.dt.month
df['month_str'] = df.date.dt.strftime('%b')
df['wind_speed_round'] = df.wind_speed.round(0)
df

,date,air_temp,wet_temp,rel_hum,wind_speed,wind_dir,wind_dir_bin,month,month_str,wind_speed_round
0,2002-08-01 00:00:00,14.4,13.6,91.0,1.5,200.0,210,8,Aug,2.0
1,2002-08-01 01:00:00,13.3,12.8,94.2,0.5,190.0,180,8,Aug,0.0
2,2002-08-01 02:00:00,13.0,12.7,96.5,0.5,200.0,210,8,Aug,0.0
3,2002-08-01 03:00:00,12.8,12.5,96.4,0.0,0.0,0,8,Aug,0.0
4,2002-08-01 04:00:00,12.4,12.2,97.6,0.0,0.0,0,8,Aug,0.0
...,...,...,...,...,...,...,...,...,...,...
175315,2022-07-31 19:00:00,24.8,18.8,53.8,3.6,320.0,330,7,Jul,4.0
175316,2022-07-31 20:00:00,23.4,18.6,61.1,4.1,10.0,0,7,Jul,4.0
175317,2022-07-31 21:00:00,21.6,17.9,68.0,4.1,50.0,60,7,Jul,4.0
175318,2022-07-31 22:00:00,20.6,17.5,72.6,4.1,60.0,60,7,Jul,4.0


In [39]:
# Check that the wind directions above 345 are in the 0 bin
assert((df[df.wind_dir > 345].wind_dir_bin == 0).all()) 

In [40]:
df_freq , df_freq_norm = frequency_tables(df)
df_freq.tail()

wind_dir_bin,0,30,60,90,120,150,180,210,240,270,300,330,ALL OBS
wind_speed_round,,,,,,,,,,,,,
49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALL OBS,13374.0,13352.0,12224.0,10204.0,6578.0,6873.0,10907.0,22203.0,28843.0,22260.0,14666.0,13134.0,174618.0


In [41]:
df_freq_norm.head()

wind_dir_bin,0,30,60,90,120,150,180,210,240,270,300,330,ALL OBS
wind_speed_round,,,,,,,,,,,,,
0,2.9,0.2,0.2,0.2,0.2,0.2,0.3,0.5,0.5,0.5,0.6,0.6,7.1
1,0.4,0.3,0.3,0.2,0.3,0.3,0.4,0.6,0.7,0.7,0.8,0.8,5.8
2,0.9,1.1,1.0,0.7,0.7,0.8,1.1,1.5,1.8,1.5,1.6,1.6,14.3
3,1.1,1.6,1.3,1.2,0.8,0.9,1.3,2.1,2.5,2.0,1.6,1.5,17.7
4,1.0,1.6,1.3,1.2,0.7,0.7,1.2,2.2,2.7,2.0,1.2,1.2,17.2


## Calculate frequency tables and write to excel

In [42]:
import shutil
import pandas as pd
import openpyxl

NEW_FILE = 'export_wind_data_openpy.xlsx'
# Copy the template file to a new file
shutil.copyfile(TEMPLATE_FILE, NEW_FILE)

# Load workbook
wb = openpyxl.load_workbook(NEW_FILE)
ws = wb['Sheet1']

# Load and format header
header_lst = read_header(WEATHER_DATA)
# Write header
for row, header in enumerate(header_lst):
    ws.cell(row=row+1, column=1).value = header

# location list from excel template (it is not uniform)
loc_list = [14, 78, 144, 208, 274, 338, 404, 468, 534, 598, 664, 728, 794, 858, 
            924, 988, 1054, 1118, 1184, 1248, 1314, 1378, 1444, 1508, 1574, 1638]

counter = 0

# Loop through each month
for month in range(1, 13):
    df_month = df[df.month == month]
    df_freq_month, df_freq_norm_month = frequency_tables(df_month)
    
    # Select initial cell and increase the row by 64 for each table for each month
    # Update workbook at specified range
    
    for rowid, row in enumerate(df_freq_month.values.tolist()):
        for col in range(len(row)):
            ws.cell(row=loc_list[counter]+(rowid), column=col+2).value = row[col]
    
    counter += 1
    
    for rowid, row in enumerate(df_freq_norm_month.values.tolist()):
        for col in range(len(row)):
            ws.cell(row=loc_list[counter]+(rowid), column=col+2).value = row[col]
    counter += 1


# Write annual data
df_freq , df_freq_norm = frequency_tables(df)
# Update workbook at specified range
for rowid, row in enumerate(df_freq.values.tolist()):
    for col in range(len(row)):
        ws.cell(row=loc_list[counter]+(rowid), column=col+2).value = row[col]

counter += 1

for rowid, row in enumerate(df_freq_norm.values.tolist()):
    for col in range(len(row)):
        ws.cell(row=loc_list[counter]+(rowid), column=col+2).value = row[col]

counter += 1

# Save and close workbook
wb.save(NEW_FILE)
wb.close()


In [43]:
# # VERSION WITH XLWINGS

# # Copy the template file to a new file
# shutil.copyfile(TEMPLATE_FILE, 'export_wind_data.xlsx')

# # Load workbook
# app = xw.App(visible=False)
# wb = xw.Book('export_wind_data.xlsx')
# ws = wb.sheets['Sheet1']

# # Load and format header
# header_lst = read_header(WEATHER_DATA)
# # Write header
# ws.range('A1').options(index=False, header=False).value = pd.Series(header_lst)

# # location list from excel template (it is not uniform)
# loc_list = [14, 78, 144, 208, 274, 338, 404, 468, 534, 598, 664, 728, 794, 858, 
#             924, 988, 1054, 1118, 1184, 1248, 1314, 1378, 1444, 1508, 1574, 1638]

# counter = 0

# # Loop through each month
# for month in range(1, 13):
#     df_month = df[df.month == month]
#     df_freq_month, df_freq_norm_month = frequency_tables(df_month)
    
#     # Select initial cell and increase the row by 64 for each table for each month
#     # Update workbook at specified range
#     ws.range(f"B{loc_list[counter]}").options(index=False, header=False).value = df_freq_month
#     counter += 1
    
#     # Update workbook at specified range
#     ws.range(f"B{loc_list[counter]}").options(index=False, header=False).value = df_freq_norm_month
#     counter += 1
    
# # Write annual data
# df_freq , df_freq_norm = frequency_tables(df)
# # Update workbook at specified range
# ws.range(f"B{loc_list[counter]}").options(index=False, header=False).value = df_freq
# counter += 1

# ws.range(f"B{loc_list[counter]}").options(index=False, header=False).value = df_freq_norm

# #Close workbook
# wb.save()
# wb.close()
# app.quit()